# 1.6 Adding resource constraints to the model

In [ ]:
# install dependencies
%pip install -q amplpy pandas

from amplpy import AMPL, ampl_notebook
import pandas as pd

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics

Processing of steel slabs is not a single operation, but a series of
steps that may proceed at different rates.  To motivate a more
general model, imagine that we divide production into a reheat stage
that can process the incoming slabs at 200 tons per hour, and a
rolling stage that makes bands, coils or plate at the rates previously
given.  Further imagine that there are only 35 hours of reheat time,
even though there are 40 hours of rolling time.

To cover this kind of situation, we can add a set
`STAGE`
of production stages to our model.  The parameter and constraint declarations are
modified accordingly, as shown in 
[Figure 1-6a](#fig-1-6a)
.
Since there is a potentially different number of hours available in
each stage, the parameter
`avail`
is now indexed over
`STAGE` .
Since there is a potentially different production rate for each
product in each stage, the parameter
`rate`
is indexed over both
`PROD`
and
`STAGE` .
In the
`Time`
constraint, the production rate for product
`p`
in stage
`s`
is referred to as
`rate[p,s]` ;
this is AMPL's version of a doubly subscripted entity like
$a_{ps}$ in algebraic notation.

<a id='fig-1-6a'><center><b>Figure 1-6a:</b> Additional resource constraints (steel4.mod).</center></a>

In [ ]:
%%writefile steel4.mod

set PROD;   # products
set STAGE;  # stages

param rate {PROD,STAGE} > 0; # tons per hour in each stage
param avail {STAGE} >= 0;    # hours available/week in each stage
param profit {PROD};         # profit per ton

param commit {PROD} >= 0;    # lower limit on tons sold in week
param market {PROD} >= 0;    # upper limit on tons sold in week

var Make {p in PROD} >= commit[p], <= market[p]; # tons produced

# Objective: total profits from all products
maximize Total_Profit: sum {p in PROD} profit[p] * Make[p];
               
# In each stage: total of hours used by all
# products may not exceed hours available
subject to Time {s in STAGE}:
   sum {p in PROD} (1/rate[p,s]) * Make[p] <= avail[s];

The only other change is to the constraint declaration, where we no
longer have a single constraint, but a constraint for each stage,
imposed by limited time available at that stage.  In algebraic notation, this might have been
written

$$
\text{Subject to}  \sum_{p \in P} (1 / a_{ps}) X_p \leq b_s, \text{ for each } s \in S.
$$

Compare the AMPL version:

```
subject to Time {s in STAGE}:
   sum {p in PROD} (1/rate[p,s]) * Make[p] <= avail[s];
```

As in the other examples, this is a straightforward analogue,
adapted to the requirements of a computer language.  In almost all
models, most of the constraints are indexed collections like this
one.

Since
`rate`
is now indexed over combinations of two indices, it
requires a data table all to itself, as in 
[Figure 1-6b](#fig-1-6b).
The data file must also include the membership for the new set
`STAGE` ,
and values of
`avail`
for both
`reheat`
and
`roll` .

<a id='fig-1-6b'><center><b>Figure 1-6b:</b> Data for additional resource constraints.</center></a>

In [3]:
df_prod = pd.DataFrame(
    [
        ['bands', 25, 1000, 6000],
        ['coils', 30, 500, 4000],
        ['plate', 29, 750, 3500]
    ],
    columns=['PROD', 'profit', 'commit', 'market']
).set_index('PROD')

df_stage = pd.DataFrame(
    [
        ['reheat', 35],
        ['roll', 40]
    ],
    columns=['STAGE', 'avail']
).set_index('STAGE')

rate = {
    ('bands', 'reheat') : 200,
    ('bands', 'roll') : 200,
    ('coils', 'reheat') : 200,
    ('coils', 'roll') : 140,
    ('plate', 'reheat') : 200,
    ('plate', 'roll') : 160  
}

df_rate_a = pd.DataFrame(
    [
        ['bands', 'reheat', 200],
        ['bands', 'roll', 200],
        ['coils', 'reheat', 200],
        ['coils', 'roll', 140],
        ['plate', 'reheat', 200],
        ['plate', 'roll', 160]
    ],
    columns=['PROD', 'STAGE', 'rate']
).set_index(['PROD', 'STAGE'])

df_rate_b = pd.DataFrame(
    [
        ['bands', 200, 200],
        ['coils', 200, 140],
        ['plate', 200, 160]
    ],
    columns=['PROD', 'reheat', 'roll']
).set_index('PROD')

df_rate_c = pd.DataFrame(
    [
        ['reheat', 200, 200, 200],
        ['roll', 200, 140, 160]
    ],
    columns=['STAGE', 'bands', 'coils', 'plate']
).set_index('STAGE').T     

display(df_prod)
display(df_stage)

print(rate)
     
display(df_rate_a)
display(df_rate_b)
display(df_rate_c)

,profit,commit,market
PROD,,,
bands,25,1000,6000
coils,30,500,4000
plate,29,750,3500


,avail
STAGE,
reheat,35
roll,40


{('bands', 'reheat'): 200, ('bands', 'roll'): 200, ('coils', 'reheat'): 200, ('coils', 'roll'): 140, ('plate', 'reheat'): 200, ('plate', 'roll'): 160}


rate
PROD  STAGE       
bands reheat   200
      roll     200
coils reheat   200
      roll     140
plate reheat   200
      roll     160

,reheat,roll
PROD,,
bands,200,200
coils,200,140
plate,200,160


STAGE,reheat,roll
bands,200,200
coils,200,140
plate,200,160


After these changes are made, we use AMPL to get another revised
solution:

In [4]:
ampl.reset()
ampl.option['solver'] = 'highs'
ampl.read('steel4.mod')
ampl.set_data(df_prod, 'PROD')
ampl.set_data(df_stage, 'STAGE')
ampl.param['rate'] = df_rate_c
ampl.solve()
ampl.display('Make.lb', 'Make', 'Make.ub', 'Make.rc')
ampl.display('Time')

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 190071.4286
2 simplex iterations
0 barrier iterations
 
:     Make.lb    Make   Make.ub     Make.rc       :=
bands   1000    3357.14   6000    -5.32907e-15
coils    500     500      4000    -1.85714
plate    750    3142.86   3500    -7.10543e-15
;

Time [*] :=
reheat  1800
  roll  3200
;



The
`reset()`
function erases the previous model so a
new one can be read in.

At the end of the example above we have displayed the "marginal values"
(also called "dual values"
or "shadow prices") associated with the
`Time`
constraints.  The marginal value of a constraint measures how much the
value of the objective would improve if the constraint were
relaxed by a small amount.  For example, here we would expect that
up to some point, additional reheat time would produce another
$\$$1800 of extra profit per hour, and additional rolling time
would produce $\$$3200 per hour; decreasing these times would
decrease the profit correspondingly.  In output commands like
`display()` ,
AMPL interprets a constraint's name alone as referring to the
associated marginal values.

We also display several quantities associated with the variables
`Make` .
First there are lower bounds
`Make.lb`
and upper bounds
`Make.ub` ,
which in this case are the same as
`commit`
and
`market` .
We also show the `reduced cost`
`Make.rc` ,
which has the same meaning with respect to the bounds that the
marginal values have with respect to the constraints.  Thus we see
that, again up to some point, each increase of a ton in the lower
bound (or commitment) for coil production should reduce profits by
about $\$$1.86; each one-ton decrease in the lower bound should
improve profits by about $\$$1.86.
The production levels for
bands and plates are between their bounds, so their reduced
costs are essentially zero
(recall that
`e-15`
means $\times 10^{-15}$), and changing their levels will
have no effect.
Bounds, marginal (or dual) values,
reduced costs and other quantities associated with variables and
constraints are explored further in Section @Display@.@disp-related@. xTODO

Comparing this session with our previous one, we see that the
additional reheat time restriction reduces profits by about
$\$$4750, and forces a substantial change in the optimal
solution:  much higher production of plate and lower production of
bands.
Moreover, the logic underlying the optimum is no longer so
obvious.  It is the difficulty of solving LPs by logical reasoning alone
that necessitates computer-based systems such as AMPL.

# Bibliography

<a id="1">[1]</a>
Julius S. Aronofsky, John M. Dutton and Michael T. Tayyabkhan,
*Managerial Planning with Linear Programming: In Process Industry Operations*.
John Wiley & Sons (New York, NY, 1978).  A detailed account of
a variety of profit-maximizing applications, with emphasis on the
petroleum and petrochemical industries.

<a id="2">[2]</a>
Va\o"s\(va"ek Chv\o"a\'"tal,
*Linear Programming* ,
W. H. Freeman (New York, NY, 1983).
A concise and economical introduction to theoretical and algorithmic topics
in linear programming.

<a id="3">[3]</a>
Tibor Fabian,
*A Linear Programming Model of Integrated Iron and Steel Production.*
Management Science
.B 4
(1958) pp. 415-449.  An
application to all stages of steelmaking - from coal and ore through
finished products - from the early days of linear programming.

<a id="4">[4]</a>
Robert Fourer and Goutam Dutta,
*A Survey of Mathematical Programming Applications in Integrated Steel Plants.*
Manufacturing & Service Operations Management
.B 4
(2001) pp. 387 - 400.

<a id="5">[5]</a>
David A. Kendrick, Alexander Meeraus and Jaime Alatorre,
*The Planning of Investment Programs in the Steel Industry*.
The Johns Hopkins University Press (Baltimore, MD, 1984).
Several detailed
mathematical programming models, using the Mexican steel industry
as an example.

<a id="6">[6]</a>
Robert J. Vanderbei,
*Linear Programming: Foundations and Extensions*
(2nd edition).  Kluwer Academic Publishers (Dordrecht, The Netherlands, 2001).
An updated survey of linear programming theory and methods.